In [1]:
import numpy as np
import trimesh
import pyrender
import matplotlib.pyplot as plt
import os
from pathlib import Path
import logging

# --- Configuration and Constants ---

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Set environment variables for headless rendering with pyrender
os.environ["PYOPENGL_PLATFORM"] = "egl"
os.environ["LIBGL_ALWAYS_SOFTWARE"] = "1"

# --- Path Definitions ---
# Note: Using pathlib for robust path management.
INPUT_ROOT = Path("/home/jeans/win/aaaJAIST/resources/LOD_data_50")
OUTPUT_ROOT = Path("/home/jeans/progressive_img2sketch/resources/LOD_images")

# --- Debugging and Rendering Settings ---
SHOW_Y_AXIS_HELPER = True  # If True, adds a tall, thin box to indicate the Y-axis.
CAMERA_ANGLES_DEG = range(0, 360, 45) # Capture every 45 degrees around the model.
CAMERA_ELEVATIONS_DEG = range(0, 16, 15) # Capture at 0 and 15 degrees elevation.


# --- Core Functions: Alignment ---

def get_registration_matrix(mesh_to_move: trimesh.Trimesh, reference_mesh: trimesh.Trimesh) -> tuple[np.ndarray, float]:
    """
    Calculates the transformation matrix to align 'mesh_to_move' to 'reference_mesh' using ICP.

    Args:
        mesh_to_move: The mesh to be transformed.
        reference_mesh: The mesh to align to.

    Returns:
        A tuple containing the 4x4 transformation matrix and the final ICP cost.
    """
    matrix, _, cost = trimesh.registration.icp(
        mesh_to_move,
        reference_mesh,
        samples=5000,
        scale=False,
        max_iterations=50
    )
    return matrix, cost

def align_and_center_lods(lod1_mesh: trimesh.Trimesh, lod2_mesh: trimesh.Trimesh, lod3_mesh: trimesh.Trimesh) -> tuple:
    """
    Aligns LODs 2 and 3 to LOD1, then centers all three meshes based on their combined bounding box.

    Args:
        lod1_mesh: The highest detail mesh (reference).
        lod2_mesh: The medium detail mesh.
        lod3_mesh: The lowest detail mesh.

    Returns:
        A tuple of the three aligned and centered trimesh.Trimesh objects (lod1, lod2, lod3).
    """
    logging.info("Aligning LOD 2 to LOD 1...")
    transform_2_to_1, cost_21 = get_registration_matrix(lod2_mesh, lod1_mesh)
    lod2_mesh.apply_transform(transform_2_to_1)
    logging.info(f"LOD 2 -> 1 alignment complete. Cost: {cost_21:.4f}")

    logging.info("Aligning LOD 3 to LOD 1...")
    transform_3_to_1, cost_31 = get_registration_matrix(lod3_mesh, lod1_mesh)
    lod3_mesh.apply_transform(transform_3_to_1)
    logging.info(f"LOD 3 -> 1 alignment complete. Cost: {cost_31:.4f}")

    # Center all three meshes together based on their combined bounds
    combined_mesh = trimesh.util.concatenate([lod1_mesh, lod2_mesh, lod3_mesh])
    center_translation = -combined_mesh.bounds.mean(axis=0)
    
    lod1_mesh.apply_translation(center_translation)
    lod2_mesh.apply_translation(center_translation)
    lod3_mesh.apply_translation(center_translation)
    logging.info("All LODs centered at the origin.")
    
    return lod1_mesh, lod2_mesh, lod3_mesh


# --- Core Functions: Rendering ---

def look_at_matrix(eye: np.ndarray, target: np.ndarray, up: np.ndarray) -> np.ndarray:
    """
    Creates a camera-to-world pose matrix for PyRender.

    Args:
        eye: The 3D position of the camera.
        target: The 3D point the camera is looking at.
        up: The 3D 'up' direction vector.

    Returns:
        A 4x4 camera pose matrix.
    """
    f = target - eye
    f = f / np.linalg.norm(f)  # Forward

    s = np.cross(f, up)
    s = s / np.linalg.norm(s)  # Right

    u = np.cross(s, f)         # Up

    pose = np.eye(4)
    pose[:3, 0] = s
    pose[:3, 1] = u
    pose[:3, 2] = -f
    pose[:3, 3] = eye
    
    return pose

def capture_orbit_views(mesh: trimesh.Trimesh, scene_number: int, lod_number: int):
    """
    Renders a mesh from various orbital camera angles and saves the images.

    Args:
        mesh: The trimesh object to render.
        scene_number: The ID of the scene (for naming files).
        lod_number: The LOD number (for naming files).
    """
    scene = pyrender.Scene(bg_color=[0.1, 0.1, 0.1, 1.0], ambient_light=[0.3, 0.3, 0.3])
    scene.add(pyrender.Mesh.from_trimesh(mesh, smooth=True))
    
    # Add a visual helper for the Y-axis if enabled
    if SHOW_Y_AXIS_HELPER:
        axis_mesh = trimesh.creation.box(extents=[0.01, mesh.extents.max() * 2, 0.01])
        scene.add(pyrender.Mesh.from_trimesh(axis_mesh, smooth=False))

    # Set up lighting
    light = pyrender.DirectionalLight(color=[1.0, 1.0, 1.0], intensity=5.0)
    scene.add(light, pose=np.eye(4))

    # Set up renderer
    renderer = pyrender.OffscreenRenderer(viewport_width=800, viewport_height=800)
    
    # Calculate camera distance based on mesh size
    radius = np.max(mesh.extents)
    camera_distance_factor = 2.0
    
    camera = pyrender.PerspectiveCamera(yfov=np.pi / 3.0, aspectRatio=1.0)
    camera_node = scene.add(camera, pose=np.eye(4))

    for angle in CAMERA_ANGLES_DEG:
        for elevation in CAMERA_ELEVATIONS_DEG:
            angle_rad = np.deg2rad(angle)
            elevation_rad = np.deg2rad(elevation)

            # Calculate camera position
            x = radius * camera_distance_factor * np.cos(elevation_rad) * np.sin(angle_rad)
            y = radius * camera_distance_factor * np.sin(elevation_rad)
            z = radius * camera_distance_factor * np.cos(elevation_rad) * np.cos(angle_rad)
            
            eye = np.array([x, y, z])
            target = np.array([0.0, 0.0, 0.0])
            up_vec = np.array([0.0, 1.0, 0.0])

            camera_pose = look_at_matrix(eye, target, up_vec)
            scene.set_pose(camera_node, pose=camera_pose)

            # Render the scene
            color, _ = renderer.render(scene)

            # Define output path and save the image
            output_dir = OUTPUT_ROOT / str(scene_number) / f"lod{lod_number}" / str(angle) / str(elevation)
            output_dir.mkdir(parents=True, exist_ok=True)
            
            filename = f"{scene_number}_{lod_number}_{angle}_{elevation}.png"
            output_path = output_dir / filename
            
            plt.imsave(output_path, color)
            logging.info(f"Saved image: {output_path}")

    renderer.delete()

# --- Main Processing Logic ---

def process_scene(scene_number: int):
    """
    Loads, aligns, and renders all LODs for a single scene.
    
    Args:
        scene_number: The ID of the scene to process.
    """
    logging.info(f"--- Processing Scene: {scene_number} ---")
    try:
        # Load all three LOD meshes
        lod1_path = INPUT_ROOT / str(scene_number) / "lod1.obj"
        lod2_path = INPUT_ROOT / str(scene_number) / "lod2.obj"
        lod3_path = INPUT_ROOT / str(scene_number) / "lod3.obj"
        
        lod1_mesh = trimesh.load(lod1_path, force='mesh')
        lod2_mesh = trimesh.load(lod2_path, force='mesh')
        lod3_mesh = trimesh.load(lod3_path, force='mesh')

        # Align and center the meshes
        lod1_aligned, lod2_aligned, lod3_aligned = align_and_center_lods(lod1_mesh, lod2_mesh, lod3_mesh)

        # Render each aligned LOD
        meshes_to_render = {1: lod1_aligned, 2: lod2_aligned, 3: lod3_aligned}
        for lod_num, mesh in meshes_to_render.items():
            logging.info(f"Rendering LOD {lod_num} for scene {scene_number}...")
            capture_orbit_views(mesh, scene_number, lod_num)

    except Exception as e:
        logging.error(f"Failed to process scene {scene_number}. Error: {e}")


def main():
    """
    Main function to iterate through the dataset and process each scene.
    """
    # Ensure the main output directory exists
    OUTPUT_ROOT.mkdir(parents=True, exist_ok=True)
    
    for scene_number in range(51):  # Process scenes 0 to 50
        process_scene(scene_number)

    logging.info("--- All scenes processed. ---")


if __name__ == '__main__':
    main()

2025-07-06 14:00:25,752 - INFO - --- Processing Scene: 0 ---
2025-07-06 14:00:25,855 - INFO - triangulating faces
2025-07-06 14:00:25,857 - INFO - triangulating faces
2025-07-06 14:00:26,026 - INFO - Aligning LOD 2 to LOD 1...
2025-07-06 14:00:26,028 - ERROR - Failed to process scene 0. Error: float() argument must be a string or a real number, not 'Trimesh'
2025-07-06 14:00:26,029 - INFO - --- Processing Scene: 1 ---
2025-07-06 14:00:26,455 - INFO - Aligning LOD 2 to LOD 1...
2025-07-06 14:00:26,455 - ERROR - Failed to process scene 1. Error: float() argument must be a string or a real number, not 'Trimesh'
2025-07-06 14:00:26,456 - INFO - --- Processing Scene: 2 ---
2025-07-06 14:00:27,025 - INFO - Aligning LOD 2 to LOD 1...
2025-07-06 14:00:27,026 - ERROR - Failed to process scene 2. Error: float() argument must be a string or a real number, not 'Trimesh'
2025-07-06 14:00:27,027 - INFO - --- Processing Scene: 3 ---
2025-07-06 14:00:27,679 - INFO - Aligning LOD 2 to LOD 1...
2025-07-

: 